In [ ]:
import pandas as pd
import polars as pl
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("new-york-city/nyc-citywide-payroll-data")
os.listdir(path)
df = pl.read_csv(path+"/Citywide_Payroll_Data__Fiscal_Year_.csv")
# Remove spaces in first (or last) slot
for _column_ in ['Pay Basis', 'Work Location Borough']:
    df = df.with_columns(pl.col(_column_).str.strip_chars(' ').alias(_column_))
#df = rt.columnsAreTimestamps(df, 'Agency Start Date')
# To Upper Case
for _column_ in ['Agency Name', 'Last Name', 'First Name', 'Mid Init', 'Work Location Borough', 'Title Description']:
    df = df.with_columns(pl.col(_column_).str.to_uppercase().alias(_column_))
# Remove "$" sign... cast to float
for _column_ in ['Base Salary', 'Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']:
    df = df.with_columns(pl.col(_column_).str.strip_chars_start('$').cast(pl.Float64).alias(_column_))
# Per Session Flag
def isPerSession(x):
    if x is None: return False
    return 'session' in x.lower() and 'per' in x.lower()
df = df.with_columns(pl.col('Title Description').str.to_lowercase().str.contains('session').alias('Per Session Pay Flag'))
for x in set(df['Title Description']):
    if isPerSession(x): print(x)
# Crude fixes for Title Description
_fix_ = {}
for x in set(df['Title Description']):
    _orig_ = x
    if x is None: x = "None"
    while x.startswith('*') or x.startswith('?'): x = x[1:]
    x = x.strip()
    _fix_[_orig_] = x
df = df.with_columns(pl.col('Title Description').replace_strict(_fix_).alias('Title Description'))
# Finer fix for Agency Name
# Produces the template below
#_len_ = 40
#for x in sorted(list(set(df['Agency Name']))):
#    _other_ = f'"{x}"'
#    _str_   = f'"{x.strip()}"'
#    print(f'{_other_:{_len_}}:({_str_:{_len_}},{_str_}),')
_fix_ = {
"ADMIN FOR CHILDREN'S SVCS     "        :("ADMIN FOR CHILDREN'S SERVICES"         ,"None",              "None"),
"ADMIN TRIALS AND HEARINGS     "        :("ADMIN TRIALS AND HEARINGS"             ,"None",              "None"),
"BOARD OF CORRECTION           "        :("BOARD OF CORRECTIONS"                  ,"None",              "None"),
"BOARD OF CORRECTIONS          "        :("BOARD OF CORRECTIONS"                  ,"None",              "None"),
"BOARD OF ELECTION             "        :("BOARD OF ELECTIONS"                    ,"None",              "None"),
"BOARD OF ELECTION POLL WORKERS"        :("BOARD OF ELECTIONS"                    ,"POLL WORKERS",      "None"),
"BOROUGH PRESIDENT-BRONX       "        :("BOROUGH PRESIDENT",                     "None"              ,"BRONX"),
"BOROUGH PRESIDENT-BROOKLYN    "        :("BOROUGH PRESIDENT",                     "None"              ,"BROOKLYN"),
"BOROUGH PRESIDENT-QUEENS      "        :("BOROUGH PRESIDENT",                     "None"              ,"QUEENS"),
"BOROUGH PRESIDENT-STATEN IS   "        :("BOROUGH PRESIDENT",                     "None"              ,"STATEN ISLAND"),
"BRONX COMMUNITY BOARD #1      "        :("COMMUNITY BOARD"                       ,"BOARD #1",          "BRONX"),
"BRONX COMMUNITY BOARD #10     "        :("COMMUNITY BOARD"                       ,"BOARD #10",         "BRONX"),
"BRONX COMMUNITY BOARD #11     "        :("COMMUNITY BOARD"                       ,"BOARD #11",         "BRONX"),
"BRONX COMMUNITY BOARD #12     "        :("COMMUNITY BOARD"                       ,"BOARD #12",         "BRONX"),
"BRONX COMMUNITY BOARD #2      "        :("COMMUNITY BOARD"                       ,"BOARD #2",          "BRONX"),
"BRONX COMMUNITY BOARD #3      "        :("COMMUNITY BOARD"                       ,"BOARD #3",          "BRONX"),
"BRONX COMMUNITY BOARD #4      "        :("COMMUNITY BOARD"                       ,"BOARD #4",          "BRONX"),
"BRONX COMMUNITY BOARD #5      "        :("COMMUNITY BOARD"                       ,"BOARD #5",          "BRONX"),
"BRONX COMMUNITY BOARD #6      "        :("COMMUNITY BOARD"                       ,"BOARD #6",          "BRONX"),
"BRONX COMMUNITY BOARD #7      "        :("COMMUNITY BOARD"                       ,"BOARD #7",          "BRONX"),
"BRONX COMMUNITY BOARD #8      "        :("COMMUNITY BOARD"                       ,"BOARD #8",          "BRONX"),
"BRONX COMMUNITY BOARD #9      "        :("COMMUNITY BOARD"                       ,"BOARD #9",          "BRONX"),
"BRONX DA       "                       :("DISTRICT ATTORNEY",                     "None"              ,"BRONX"),
"BRONX DISTRICT ATTORNEY       "        :("DISTRICT ATTORNEY",                     "None"              ,"BRONX"),
"BROOKLYN COMMUNITY BOARD #1   "        :("COMMUNITY BOARD"                       ,"BOARD #1",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #10  "        :("COMMUNITY BOARD"                       ,"BOARD #10",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #11  "        :("COMMUNITY BOARD"                       ,"BOARD #11",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #12  "        :("COMMUNITY BOARD"                       ,"BOARD #12",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #13  "        :("COMMUNITY BOARD"                       ,"BOARD #13",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #14  "        :("COMMUNITY BOARD"                       ,"BOARD #14",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #15  "        :("COMMUNITY BOARD"                       ,"BOARD #15",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #16  "        :("COMMUNITY BOARD"                       ,"BOARD #16",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #17  "        :("COMMUNITY BOARD"                       ,"BOARD #17",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #18  "        :("COMMUNITY BOARD"                       ,"BOARD #18",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #2   "        :("COMMUNITY BOARD"                       ,"BOARD #2",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #3   "        :("COMMUNITY BOARD"                       ,"BOARD #3",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #4   "        :("COMMUNITY BOARD"                       ,"BOARD #4",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #5   "        :("COMMUNITY BOARD"                       ,"BOARD #5",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #6   "        :("COMMUNITY BOARD"                       ,"BOARD #6",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #7   "        :("COMMUNITY BOARD"                       ,"BOARD #7",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #8   "        :("COMMUNITY BOARD"                       ,"BOARD #8",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #9   "        :("COMMUNITY BOARD"                       ,"BOARD #9",          "BROOKLYN"),
"BUSINESS INTEGRITY COMMISSION "        :("BUSINESS INTEGRITY COMMISSION"         ,"None",              "None"),
"CAMPAIGN FINANCE BOARD        "        :("CAMPAIGN FINANCE BOARD"                ,"None",              "None"),
"CITY CLERK                    "        :("CITY CLERK"                            ,"None",              "None"),
"CITY COUNCIL                  "        :("CITY COUNCIL"                          ,"None",              "None"),
"CIVIL SERVICE COMMISSION      "        :("CIVIL SERVICE COMMISSION"              ,"None",              "None"),
"CIVILIAN COMPLAINT REVIEW BD  "        :("CIVILIAN COMPLAINT REVIEW BOARD"       ,"None",              "None"),
"COMMUNITY COLLEGE (BRONX)     "        :("COMMUNITY COLLEGE"                     ,"None"              ,"BRONX"),
"COMMUNITY COLLEGE (HOSTOS)    "        :("COMMUNITY COLLEGE"                     ,"None"              ,"HOSTOS"),
"COMMUNITY COLLEGE (KINGSBORO) "        :("COMMUNITY COLLEGE"                     ,"None"              ,"KINGSBORO"),
"COMMUNITY COLLEGE (LAGUARDIA) "        :("COMMUNITY COLLEGE"                     ,"None"              ,"LAGUARDIA"),
"COMMUNITY COLLEGE (MANHATTAN) "        :("COMMUNITY COLLEGE"                     ,"None"              ,"MANHATTAN"),
"COMMUNITY COLLEGE (QUEENSBORO)"        :("COMMUNITY COLLEGE"                     ,"None"              ,"QUEENSBORO"),
"CONFLICTS OF INTEREST BOARD   "        :("CONFLICTS OF INTEREST BOARD"           ,"None",              "None"),
"CONSUMER AFFAIRS              "        :("CONSUMER AFFAIRS"                      ,"None",              "None"),
"CULTURAL AFFAIRS              "        :("CULTURAL AFFAIRS"                      ,"None",              "None"),
"CUNY CENTRAL OFFICE           "        :("CUNY CENTRAL OFFICE"                   ,"None",              "None"),
"DEPARTMENT FOR THE AGING      "        :("DEPARTMENT FOR THE AGING"              ,"None",              "None"),
"DEPARTMENT OF BUILDINGS       "        :("DEPARTMENT OF BUILDINGS"               ,"None",              "None"),
"DEPARTMENT OF BUSINESS SERV.  "        :("DEPARTMENT OF BUSINESS SERVICES"       ,"None",              "None"),
"DEPARTMENT OF CITY PLANNING   "        :("DEPARTMENT OF CITY PLANNING"           ,"None",              "None"),
"DEPARTMENT OF CORRECTION      "        :("DEPARTMENT OF CORRECTION"              ,"None",              "None"),
"DEPARTMENT OF EDUCATION ADMIN "        :("DEPARTMENT OF EDUCATION"               ,"Admin",             "None"),
"DEPARTMENT OF FINANCE         "        :("DEPARTMENT OF FINANCE"                 ,"None",              "None"),
"DEPARTMENT OF INVESTIGATION   "        :("DEPARTMENT OF INVESTIGATION"           ,"None",              "None"),
"DEPARTMENT OF JUVENILE JUSTICE"        :("DEPARTMENT OF JUVENILE JUSTICE"        ,"None",              "None"),
"DEPARTMENT OF PROBATION       "        :("DEPARTMENT OF PROBATION"               ,"None",              "None"),
"DEPARTMENT OF SANITATION      "        :("DEPARTMENT OF SANITATION"              ,"None",              "None"),
"DEPARTMENT OF TRANSPORTATION  "        :("DEPARTMENT OF TRANSPORTATION"          ,"None",              "None"),
"DEPT OF CITYWIDE ADMIN SVCS   "        :("DEPT OF CITYWIDE ADMIN SERVICES"       ,"None",              "None"),
"DEPT OF ED HRLY SUPPORT STAFF "        :("DEPT OF EDUCATION"                     ,"HOURLY SUPPORT STAFF", "None"),
"DEPT OF ED PARA PROFESSIONALS "        :("DEPT OF EDUCATION"                     ,"PARA PROFESSIONALS",   "None"),
"DEPT OF ED PEDAGOGICAL        "        :("DEPT OF EDUCATION"                     ,"PEDAGOGICAL",          "None"),
"DEPT OF ED PER DIEM TEACHERS  "        :("DEPT OF EDUCATION"                     ,"PER DIEM TEACHERS",    "None"),
"DEPT OF ED PER SESSION TEACHER"        :("DEPT OF EDUCATION"                     ,"PER SESSION TEACHER",  "None"),
"DEPT OF ENVIRONMENT PROTECTION"        :("DEPT OF ENVIRONMENT PROTECTION"        ,"None",                 "None"),
"DEPT OF HEALTH/MENTAL HYGIENE "        :("DEPT OF HEALTH/MENTAL HYGIENE"         ,"None",                 "None"),
"DEPT OF INFO TECH & TELECOMM  "        :("DEPT OF INFO TECH & TELECOMM"          ,"None",                 "None"),
"DEPT OF PARKS & RECREATION    "        :("DEPT OF PARKS & RECREATION"            ,"None",                 "None"),
"DEPT OF RECORDS & INFO SERVICE"        :("DEPT OF RECORDS & INFO SERVICE"        ,"None",                 "None"),
"DEPT OF YOUTH & COMM DEV SRVS "        :("DEPT OF YOUTH & COMM DEV SRVS"         ,"None",                 "None"),
"DEPT. OF DESIGN & CONSTRUCTION"        :("DEPT OF DESIGN & CONSTRUCTION"         ,"None",                 "None"),
"DEPT. OF HOMELESS SERVICES    "        :("DEPT OF HOMELESS SERVICES"             ,"None",                 "None"),
"DISTRICT ATTORNEY KINGS COUNTY"        :("DISTRICT ATTORNEY"                     ,"None"                 ,"BROOKLYN"), # apparently, kings county is actually brooklyn
"DISTRICT ATTORNEY QNS COUNTY  "        :("DISTRICT ATTORNEY"                     ,"None"                 ,"QNS COUNTY"),
"DISTRICT ATTORNEY RICHMOND COU"        :("DISTRICT ATTORNEY"                     ,"None"                 ,"RICHMOND COUNTY"),
"DISTRICT ATTORNEY-MANHATTAN   "        :("DISTRICT ATTORNEY"                     ,"None"                 ,"MANHATTAN"),
"DISTRICT ATTORNEY-SPECIAL NARC"        :("DISTRICT ATTORNEY"                     ,"None"                 ,"SPECIAL NARC"),
"DISTRICTING COMMISSION        "        :("DISTRICTING COMMISSION"                ,"None"                 ,"None"),
"DOE CUSTODIAL PAYROL          "        :("DOE CUSTODIAL PAYROL"                  ,"None"                 ,"None"),
"EQUAL EMPLOY PRACTICES COMM   "        :("EQUAL EMPLOY PRACTICES COMMISSION"     ,"None"                 ,"None"),
"FINANCIAL INFO SVCS AGENCY    "        :("FINANCIAL INFO SERVICES AGENCY"        ,"None"                 ,"None"),
"FIRE DEPARTMENT               "        :("FIRE DEPARTMENT"                       ,"None"                 ,"None"),
"GUTTMAN COMMUNITY COLLEGE     "        :("GUTTMAN COMMUNITY COLLEGE"             ,"None"                 ,"None"),
"HOUSING PRESERVATION & DVLPMNT"        :("HOUSING PRESERVATION & DEVELOPMENT"    ,"None"                 ,"None"),
"HRA/DEPT OF SOCIAL SERVICES   "        :("DEPT OF SOCIAL SERVICES"               ,"HRA"                  ,"None"),
"HUMAN RIGHTS COMMISSION       "        :("HUMAN RIGHTS COMMISSION"               ,"None"                 ,"None"),
"HUNTER COLLEGE HIGH SCHOOL    "        :("HUNTER COLLEGE HIGH SCHOOL"            ,"None"                 ,"None"),
"INDEPENDENT BUDGET OFFICE     "        :("INDEPENDENT BUDGET OFFICE"             ,"None"                 ,"None"),
"KINGS DA"                              :("DISTRICT ATTORNEY"                     ,"None"                 ,"BROOKLYN"), # apparently, kings county is actually brooklyn
"LANDMARKS PRESERVATION COMM   "        :("LANDMARKS PRESERVATION COMM"           ,"None"                 ,"None"),
"LAW DEPARTMENT                "        :("LAW DEPARTMENT"                        ,"None"                 ,"None"),
"MANHATTAN COMMUNITY BOARD #1  "        :("COMMUNITY BOARD"                       ,"BOARD #1"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #10 "        :("COMMUNITY BOARD"                       ,"BOARD #10"            ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #11 "        :("COMMUNITY BOARD"                       ,"BOARD #11"            ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #12 "        :("COMMUNITY BOARD"                       ,"BOARD #12"            ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #2  "        :("COMMUNITY BOARD"                       ,"BOARD #2"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #3  "        :("COMMUNITY BOARD"                       ,"BOARD #3"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #4  "        :("COMMUNITY BOARD"                       ,"BOARD #4"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #5  "        :("COMMUNITY BOARD"                       ,"BOARD #5"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #6  "        :("COMMUNITY BOARD"                       ,"BOARD #6"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #7  "        :("COMMUNITY BOARD"                       ,"BOARD #7"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #8  "        :("COMMUNITY BOARD"                       ,"BOARD #8"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #9  "        :("COMMUNITY BOARD"                       ,"BOARD #9"             ,"MANHATTAN"),
"MANHATTAN DA   "                       :("DISTRICT ATTORNEY"                     ,"None"                 ,"MANHATTAN"),
"MAYORS OFFICE OF CONTRACT SVCS"        :("MAYORS OFFICE OF CONTRACT SERVICES"    ,"None"                 ,"None"),
"MUNICIPAL WATER FIN AUTHORITY "        :("MUNICIPAL WATER FINANCIAL AUTHORITY"   ,"None"                 ,"None"),
"NYC DEPT OF VETERANS' SERVICES"        :("DEPT OF VETERANS' SERVICES"            ,"None"                 ,"None"),
"NYC EMPLOYEES RETIREMENT SYS  "        :("EMPLOYEES RETIREMENT SYSTEM"           ,"None"                 ,"None"),
"NYC HOUSING AUTHORITY         "        :("HOUSING AUTHORITY"                     ,"None"                 ,"None"),
"NYC POLICE PENSION FUND       "        :("POLICE DEPARTMENT}"                    ,"PENSION FUND"         ,"None"),
"OFF OF PAYROLL ADMINISTRATION "        :("OFFICE OF PAYROLL ADMINISTRATION"      ,"None"                 ,"None"),
"OFFICE OF COLLECTIVE BARGAININ"        :("OFFICE OF COLLECTIVE BARGAININ"        ,"None"                 ,"None"),
"OFFICE OF EMERGENCY MANAGEMENT"        :("OFFICE OF EMERGENCY MANAGEMENT"        ,"None"                 ,"None"),
"OFFICE OF LABOR RELATIONS     "        :("OFFICE OF LABOR RELATIONS"             ,"None"                 ,"None"),
"OFFICE OF MANAGEMENT & BUDGET "        :("OFFICE OF MANAGEMENT & BUDGET"         ,"None"                 ,"None"),
"OFFICE OF THE ACTUARY         "        :("OFFICE OF THE ACTUARY"                 ,"None"                 ,"None"),
"OFFICE OF THE COMPTROLLER     "        :("OFFICE OF THE COMPTROLLER"             ,"None"                 ,"None"),
"OFFICE OF THE MAYOR           "        :("OFFICE OF THE MAYOR"                   ,"None"                 ,"None"),
"PERSONNEL MONITORS            "        :("PERSONNEL MONITORS"                    ,"None"                 ,"None"),
"POLICE DEPARTMENT"                     :("POLICE DEPARTMENT"                     ,"None"                 ,"None"),
"POLICE DEPARTMENT             "        :("POLICE DEPARTMENT"                     ,"None"                 ,"None"),
"PRESIDENT BOROUGH OF MANHATTAN"        :("PRESIDENT BOROUGH"                     ,"None"                 ,"MANHATTAN"),
"PUBLIC ADMINISTRATOR-BRONX    "        :("PUBLIC ADMINISTRATOR"                  ,"None"                 ,"BRONX"),
"PUBLIC ADMINISTRATOR-KINGS    "        :("PUBLIC ADMINISTRATOR"                  ,"None"                 ,"BROOKLYN"),
"PUBLIC ADMINISTRATOR-NEW YORK "        :("PUBLIC ADMINISTRATOR"                  ,"None"                 ,"None"),     # Not sure what to do here... do they mean all of NYC?
"PUBLIC ADMINISTRATOR-QUEENS   "        :("PUBLIC ADMINISTRATOR"                  ,"None"                 ,"QUEENS"),
"PUBLIC ADMINISTRATOR-RICHMOND "        :("PUBLIC ADMINISTRATOR"                  ,"None"                 ,"RICHMOND"),
"PUBLIC ADVOCATE               "        :("PUBLIC ADVOCATE"                       ,"None"                 ,"None"),
"PUBLIC SERVICE CORPS          "        :("PUBLIC SERVICE CORPS"                  ,"None"                 ,"None"),
"QUEENS COMMUNITY BOARD #1     "        :("COMMUNITY BOARD"                       ,"BOARD #1",             "QUEENS"),
"QUEENS COMMUNITY BOARD #10    "        :("COMMUNITY BOARD"                       ,"BOARD #10",            "QUEENS"),
"QUEENS COMMUNITY BOARD #11    "        :("COMMUNITY BOARD"                       ,"BOARD #11",            "QUEENS"),
"QUEENS COMMUNITY BOARD #12    "        :("COMMUNITY BOARD"                       ,"BOARD #12",            "QUEENS"),
"QUEENS COMMUNITY BOARD #13    "        :("COMMUNITY BOARD"                       ,"BOARD #13",            "QUEENS"),
"QUEENS COMMUNITY BOARD #14    "        :("COMMUNITY BOARD"                       ,"BOARD #14",            "QUEENS"),
"QUEENS COMMUNITY BOARD #2     "        :("COMMUNITY BOARD"                       ,"BOARD #2",             "QUEENS"),
"QUEENS COMMUNITY BOARD #3     "        :("COMMUNITY BOARD"                       ,"BOARD #3",             "QUEENS"),
"QUEENS COMMUNITY BOARD #4     "        :("COMMUNITY BOARD"                       ,"BOARD #4",             "QUEENS"),
"QUEENS COMMUNITY BOARD #5     "        :("COMMUNITY BOARD"                       ,"BOARD #5",             "QUEENS"),
"QUEENS COMMUNITY BOARD #6     "        :("COMMUNITY BOARD"                       ,"BOARD #6",             "QUEENS"),
"QUEENS COMMUNITY BOARD #7     "        :("COMMUNITY BOARD"                       ,"BOARD #7",             "QUEENS"),
"QUEENS COMMUNITY BOARD #8     "        :("COMMUNITY BOARD"                       ,"BOARD #8",             "QUEENS"),
"QUEENS COMMUNITY BOARD #9     "        :("COMMUNITY BOARD"                       ,"BOARD #9",             "QUEENS"),
"QUEENS DA  "                           :("DISTRICT ATTORNEY"                     ,"None"                 ,"QUEENS"),
"RICHMOND DA"                           :("DISTRICT ATTORNEY"                     ,"None"                 ,"RICHMOND"),
"SPEC NARCS-DA"                         :("DISTRICT ATTORNEY"                     ,"SPEC NARCS",           "None"),
"STATEN ISLAND COMMUNITY BD #1 "        :("COMMUNITY BOARD"                       ,"BOARD #1",             "STATEN ISLAND"),
"STATEN ISLAND COMMUNITY BD #2 "        :("COMMUNITY BOARD"                       ,"BOARD #2",             "STATEN ISLAND"),
"STATEN ISLAND COMMUNITY BD #3 "        :("COMMUNITY BOARD"                       ,"BOARD #3",             "STATEN ISLAND"),
"TAX COMMISSION                "        :("TAX COMMISSION"                        ,"None",                 "None"),
"TAXI & LIMOUSINE COMMISSION   "        :("TAXI & LIMOUSINE COMMISSION"           ,"None",                 "None"),
"TEACHERS RETIREMENT SYSTEM    "        :("DEPT OF EDUCATION"                     ,"TEACHERS RETIREMENT SYSTEM", "None"),
}
for i in range(3):
    if i == 0: _alias_ = "Agency Name Primary"
    if i == 1: _alias_ = "Agency Name Secondary"
    if i == 2: _alias_ = "Agency Name Borough"
    _lu_ = {}
    for x in _fix_.keys(): _lu_[x] = _fix_[x][i]
    for x in df['Agency Name']: 
        if x not in _lu_.keys(): raise Exception(x) # Make sure everything is covered
    df = df.with_columns(pl.col('Agency Name').replace_strict(_lu_).alias(_alias_))
print(f'{len(df):_}')
print(df.columns)

In [ ]:
#_parms_ = {'color_by':'Pay Basis'}
#_parms_ = {'color_by':'Per Session Pay Flag', 'w':256, 'h':512}
#_tiles_ = []
#_list_  = list(df.columns)
#_list_  = ['Pay Basis', 'Title Description', 'Agency Name', 'Base Salary']
#for i in range(len(_list_)): _tiles_.append(rt.histogram(df.filter(pl.col('Pay Basis') == 'per Day'), bin_by=_list_[i], **_parms_))
#rt.tile(_tiles_)

In [ ]:
#rt.xy(df, x_field='Agency Name Primary', y_field='Base Salary', w=512, color_by='Pay Basis')